In [ ]:
%cd D:\Project\Toolkit_for_Preprocessing_MXH\VSMEC_chunking

from IPython.display import clear_output
clear_output()

In [2]:
import tensorflow as tf
import torch

tf.keras.backend.clear_session()
# clear gpu memory using torch
torch.cuda.empty_cache()
# clear output
clear_output()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
train_path = (r"Data\train.csv")
dev_path = (r"Data\dev.csv")
test_path = (r"Data\test.csv")
model_name = 'vinai/phobert-base'
test_index = 20 # default None value
batch_size = 32
max_len = 128
lr = 5e-6
epochs = 2
shuffle = False

In [4]:
from Code.Dataset import split_path, create_dataloader

if test_index != None and test_index > 3:
    # Load the data
    train_path, dev_path, test_path = split_path(test_path, test_index, train_path, dev_path, test_path)
elif test_index != None: 
    print("Test index out of range. Please provide a valid interger index greater than 3.")


Training set: 14 samples
Test set: 3 samples
Development set: 3 samples


In [5]:
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer, AutoModel
)


classes = ['Anger','Disgust','Enjoyment','Fear','Other','Sadness','Surprise']
tokenizer = AutoTokenizer.from_pretrained(model_name)
embedding_model = AutoModel.from_pretrained(model_name)
classification_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(classes)  # Number of classes
)
clear_output()

# Adjust the token embeddings size if needed
embedding_model.resize_token_embeddings(len(tokenizer))


Embedding(64001, 768, padding_idx=1)

In [ ]:
import torch.nn as nn
class MultiTaskModel(nn.Module):
    def __init__(self, embedding_model, classification_model, dropout_rate=0.1):
        super(MultiTaskModel, self).__init__()
        self.embedding_model = embedding_model
        # self.classification_head = classification_model
        self.classification_head = nn.Linear(768, 1)
        self.dropout = nn.Dropout(dropout_rate)


    def forward(self, input_ids, attention_mask):
        # Check for NaN values in input tensors
        if torch.isnan(input_ids).any() or torch.isinf(input_ids).any():
            raise ValueError("Input IDs contain NaN or Inf values.")
        if torch.isnan(attention_mask).any() or torch.isinf(attention_mask).any():
            raise ValueError("Attention mask contains NaN or Inf values.")

        # Flatten inputs
        batch_size = input_ids.size(0)
        max_length = input_ids.size(1)
        input_ids_flat = input_ids.view(-1, input_ids.size(-1))
        attention_mask_flat = attention_mask.view(-1, attention_mask.size(-1))

        
        print("input_ids_flat: ", input_ids_flat.shape)
        print("attention_mask_flat: ", attention_mask_flat.shape)
        # Get embeddings
        
        # Ensure token_type_ids are passed
        token_type_ids = torch.zeros_like(input_ids_flat)  # Default to zeros if not provided

#         outputs = self.embedding_model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)


        if torch.isnan(input_ids_flat).any() or torch.isinf(input_ids_flat).any():
            raise ValueError("Input IDs contain NaN or Inf values.")
        if torch.isnan(attention_mask_flat).any() or torch.isinf(attention_mask_flat).any():
            raise ValueError("Attention mask contains NaN or Inf values.")

        outputs = self.embedding_model(input_ids=input_ids_flat, attention_mask=attention_mask_flat, token_type_ids=token_type_ids)
        


        # Check the type and contents of outputs
        print("Outputs from embedding model:", outputs)

        # Ensure outputs is in the expected format
        if isinstance(outputs, tuple):
            last_hidden_state = outputs[0]  # The first element is often last_hidden_state
        else:
            last_hidden_state = outputs.last_hidden_state if hasattr(outputs, 'last_hidden_state') else None

        # Check if last_hidden_state is None
        if last_hidden_state is None:
            print("last_hidden_state is None. Check the output structure of the embedding model.")

        # Debugging output for last_hidden_state
        print("Last hidden state shape:", last_hidden_state.shape)

        # Check for NaNs in last_hidden_state
        if torch.isnan(last_hidden_state).any():
            print("last_hidden_state values:", last_hidden_state)
            print("last_hidden_state contains NaN values.")

        if outputs is None:
            print("Embedding model output is None.")
        # Mean pooling
        print("last_hidden_state: ", last_hidden_state.shape)
        
        
        if torch.isnan(last_hidden_state).any():
            print("last_hidden_state contain NaN values. ")
        pooled_embeddings = last_hidden_state.mean(dim=1)
        if torch.isnan(pooled_embeddings).any():
            print("Pooled embeddings contain NaN values.")
        pooled_embeddings = pooled_embeddings.view(batch_size, max_length, 768).mean(dim=1)
        print("pooled_embeddings: ", pooled_embeddings.shape)
        
        
        if torch.isnan(pooled_embeddings).any():
            print("Pooled embeddings contain NaN values. 1")

        pooled_output = self.dropout(pooled_embeddings)
        print("pooled_output: ", pooled_output.shape)

        if torch.isnan(pooled_output).any():
            print("Pooled output contains NaN values after dropout.")
            
        

        # Logits computation
        logits = self.classification_head(pooled_output)
#         if logits is None:
#             raise ValueError("Logits are None after classification head.")

        print("Logits: ", logits)

        logits = torch.sigmoid(logits)  # Adjust if necessary for your task
        print("Logits: ", logits)
        return logits






In [6]:
train_dataloader = create_dataloader(train_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=False)
dev_dataloader = create_dataloader(dev_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=False)
test_dataloader = create_dataloader(test_path, batch_size=batch_size, tokenizer = tokenizer, max_len=max_len, shuffle=False)

# Get the first batch of data from the DataLoader
first_batch = next(iter(test_dataloader))

# Access input_ids, attention_mask, and labels
input_ids = first_batch['input_ids']
attention_mask = first_batch['attention_mask']
labels = first_batch['label']

# Print to check
print(f"Input IDs: {input_ids.size()}")
print(f"Attention Mask: {attention_mask.size()}")
print(f"Labels: {labels.shape}")

Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Input IDs: torch.Size([2, 4, 128])
Attention Mask: torch.Size([2, 4, 128])
Labels: torch.Size([2, 7])


In [7]:
# Set up the model
from Code.Model import setup_model, train, test , MultiTaskModel

model, criterion, optimizer, device, num_epochs = setup_model(
    model_class = MultiTaskModel, 
    embedding_model = embedding_model, 
    classification_model = classification_model,
    lr=5e-6,
    weight_decay=1e-5,
    num_epochs=2
    )

In [8]:
# Assuming you've set up the model, dataloaders, criterion, and optimizer
train(model, train_dataloader, dev_dataloader, criterion, optimizer, device, num_epochs)

Epoch: 1


Training Loss: 0.6940
Validation Loss: 0.6590
Macro F1-Score: 0.1429
Epoch: 2


Training Loss: 0.6695
Validation Loss: 0.6478
Macro F1-Score: 0.1429


In [9]:

predictions, true_labels = test(model, test_dataloader, device)

Testing: 100%|██████████| 1/1 [00:01<00:00,  1.84s/it]

Macro F1 Score: 0.4400


In [3]:
import torch
import transformers
print(torch.__version__)
print(transformers.__version__)


c:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.0.1+cu117
4.44.0


In [2]:
!python main.py --model "vinai/phobert-base" --train_path "Data\train.csv" --dev_path "Data\dev.csv" --test_path "Data\test.csv" --batch_size 8 --max_len 128 --lr 5e-6 --num_epochs 2 --output_json "test_results.json" --output_dir "output" --test_index 50

Using device: cuda
Training set: 35 samples
Test set: 7 samples
Development set: 8 samples
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Input IDs: torch.Size([6, 4, 128])
Attention Mask: torch.Size([6, 4, 128])
Labels: torch.Size([6, 7])
Epoch: 1
Training Loss: 0.9010
Validation Loss: 0.8919
Macro F1-Score: 0.0000
Epoch: 2
Training Loss: 0.8865
Validation Loss: 0.8701
Macro F1-Score: 0.0000
Macro F1 Score: 0.4615
Test results saved to test_results.json
Model saved to output\trained_model.pth


C:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bi

In [2]:
!python main.py --model "vinai/phobert-base" --train_path "Data\train.csv" --dev_path "Data\dev.csv" --test_path "Data\test.csv" --batch_size 8 --max_len 128 --lr 5e-6 --num_epochs 2 --output_json "test_results.json" --output_dir "output" --test_index 50

Using device: cuda
Training set: 35 samples
Test set: 7 samples
Development set: 8 samples
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Columns: Index(['index', 'Emotion', 'Text', 'Tag', 'sentence_id'], dtype='object')
Input IDs: torch.Size([6, 4, 128])
Attention Mask: torch.Size([6, 4, 128])
Labels: torch.Size([6, 7])
Epoch: 1
Training Loss: 0.8970
Validation Loss: 0.8906
Macro F1-Score: 0.0765
Epoch: 2
Training Loss: 0.8882
Validation Loss: 0.8775
Macro F1-Score: 0.0000
Macro F1 Score: 0.4545
Test results saved to test_results.json
Model saved to output\trained_model.pth


C:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
C:\Users\ndp17\.pyenv\pyenv-win\versions\3.10.5\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bi